## Boundary motif enrichment

**Versions:**  
Homer: 4.11   
bedtools: 2.29.0

For the background fasta file, I used the HiCexplorer output file _.bedgraph_ with all 5000bp increments the genome was broken up into to to generate TAD separation scores. 

In [ ]:
#target file - dtri boundaries - find fasta for coordinates
bedtools getfasta -fi /2.fasta -bed /dmel_dtri_merge/dtri_merge_mid5000 > /dmel_dtri_merge/dtri_boundaries_merge_mid5000_fasta
#background file - dtri genome - find fasta for coordinates
bedtools getfasta -fi /2.fasta -bed /2_merge_score.bedgraph > dtri_score.bedgraph_to_fasta
#homer - find whether HiC explorer boundaries are enriched for certain motifs. 
findMotifs.pl dtri_boundaries_merge_fasta fly-o out/ -fasta dtri_score.bedgraph_to_fasta

In [ ]:
#dmel boundaries
bedtools getfasta -fi /11.fasta -bed /dmel_dtri_merge/dmel_merge_mid5000 > /dmel_dtri_merge/dmel_boundaries_merge_mid5000_fasta
bedtools getfasta -fi /11.fasta -bed /11_merge_score.bedgraph > dmel_score.bedgraph_to_fasta
findMotifs.pl dmel_boundaries_merge_fasta fly-o out/ -fasta dmel_score.bedgraph_to_fasta


#### Motif enrichment at novel boundaries

Target files: boundary locations in one species that are not a boundary in the other.   
Background files: orthologous genomic location in other species that is NOT a boundary.  

In [ ]:
findMotifs.pl dtri_to_dmel_dtri_nonconserved_coords_fasta fly-o dtri_to_dmel_out_cvnc/ -fasta dtri_to_dmel_dmel_non_boundary_coords_fasta
findMotifs.pl dmel_to_dtri_dmel_nonconserved_coords_fasta fly-o dmel_to_dtri_out_cvnc/ -fasta dmel_to_dtri_dtri_non_boundary_coord_fasta

#### Motif enrichment at novel boundaries  vs orthologous region in other species (Table S5)

Master.sh

In [ ]:
#find sequence of dtri lineage-specific boundaries and orthologous dmel regions 
bedtools getfasta -fi dtri_ref.fasta -bed dtri_to_dmel_dtri_nonconserved_coords_with_bound_id > dtri_to_dmel_dtri_nonconserved_coords_with_bound_id_fasta
bedtools getfasta -fi dmel_ref.fasta -bed dtri_to_dmel_dmel_nonconserved_coords_with_bound_id > dtri_to_dmel_dmel_nonconserved_coords_with_bound_id_fasta
#find sequence of dtri lineage-specific boundaries and orthologous dmel regions 
bedtools getfasta -fi dtri_ref.fasta -bed dmel_to_dtri_dtri_non_boundary_coord_with_bound_id > dmel_to_dtri_dtri_non_boundary_coord_with_bound_id_fasta
bedtools getfasta -fi dmel_ref.fasta -bed dmel_to_dtri_dmel_nonconserved_coords_with_bound_id > dmel_to_dtri_dmel_nonconserved_coords_with_bound_id_fasta

sh paired_comp.dtri_beaf.sh
sh paired_comp.dtri_m1bp.sh
cat t4.dtri.beaf t4.dtri.m1bp | sort -k1,1 | bedtools groupby -g 1 -c 2,3,4 -o sum,sum,distinct > dtri.out

sh paired_comp.dmel_beaf.sh
sh paired_comp.dmel_m1bp.sh
cat t4.dmel.beaf t4.dmel.m1bp | sort -k1,1 | bedtools groupby -g 1 -c 2,3,4 -o sum,sum,distinct > dmel.out

cat dmel.out dtri.out | awk '$2>0 || $3>0' > combined_boundary_vs_ortholog.motif_counts

paired_comp.dtri_beaf.sh

In [ ]:
#find motifs in dtri lineage-specific boundaries and orthologous dmel regions 
./bin/fimo --thresh 1e-5 --text BEAF32_MA0529.2.meme dtri_to_dmel_dtri_nonconserved_coords_with_bound_id_fasta | grep -v motif_alt_id | awk '{print $3"\t"$4-1"\t"$5}' | sort -k1,1 -k2,2n | bedtools merge -i - | cut -f 1 | uniq -c | awk '{print $2"\t"$1}' > t1

grep ">" dtri_to_dmel_dtri_nonconserved_coords_with_bound_id_fasta | sed -r 's/>//' | awk '{print $1"\t0"}' >> t1

cat t1 | sort -k1,1 | bedtools groupby -g 1 -c 2 -o sum | sort -k1b,1 > t1.1

./bin/fimo --thresh 1e-5 --text BEAF32_MA0529.2.meme dtri_to_dmel_dmel_non_boundary_coord_with_bound_id_fasta | grep -v motif_alt_id | awk '{print $3"\t"$4-1"\t"$5}' | sort -k1,1 -k2,2n | bedtools merge -i - | cut -f 1 | sed -r 's/Z[0-9]//' | sort | uniq -c | awk '{print $2"\t"$1}' > t2

grep ">" dtri_to_dmel_dmel_non_boundary_coord_with_bound_id_fasta | sed -r 's/>//' | awk '{print $1"\t0"}' >> t2

cat t2 | sort -k1,1 | bedtools groupby -g 1 -c 2 -o sum | sort -k1b,1 > t2.1

join t1.1 t2.1 | tr " " "\t" > t3

join t3 dtri_to_dmel_dmel_non_boundary_coord_with_bound_id_fasta.len | tr " " "\t" > t4.dtri.beaf

paired_comp.dtri_m1bp.sh

In [ ]:
#find motifs in dtri lineage-specific boundaries and orthologous dmel regions 
./bin/fimo --thresh 1e-5 --text M1BP_MA1459.1.meme dtri_to_dmel_dtri_nonconserved_coords_with_bound_id_fasta | grep -v motif_alt_id | awk '{print $3"\t"$4-1"\t"$5}' | sort -k1,1 -k2,2n | bedtools merge -i - | cut -f 1 | uniq -c | awk '{print $2"\t"$1}' > t1

grep ">" dtri_to_dmel_dtri_nonconserved_coords_with_bound_id_fasta | sed -r 's/>//' | awk '{print $1"\t0"}' >> t1

cat t1 | sort -k1,1 | bedtools groupby -g 1 -c 2 -o sum | sort -k1b,1 > t1.1

./bin/fimo --thresh 1e-5 --text M1BP_MA1459.1.meme dtri_to_dmel_dmel_non_boundary_coord_with_bound_id_fasta.rename | grep -v motif_alt_id | awk '{print $3"\t"$4-1"\t"$5}' | sort -k1,1 -k2,2n | bedtools merge -i - | cut -f 1 | sed -r 's/Z[0-9]//' | sort | uniq -c | awk '{print $2"\t"$1}' > t2

grep ">" dtri_to_dmel_dmel_non_boundary_coord_with_bound_id_fasta | sed -r 's/>//' | awk '{print $1"\t0"}' >> t2

cat t2 | sort -k1,1 | bedtools groupby -g 1 -c 2 -o sum | sort -k1b,1 > t2.1

join t1.1 t2.1 | tr " " "\t" > t3

join t3 dtri_to_dmel_dmel_non_boundary_coord_with_bound_id_fasta.len | tr " " "\t" > t4.dtri.m1bp

paired_comp.dmel_beaf.sh

In [ ]:
#find motifs in dmel lineage-specific boundaries and orthologous dtri regions 
./bin/fimo --thresh 1e-5 --text BEAF32_MA0529.2.meme dmel_to_dtri_dmel_nonconserved_coords_with_bound_id_fasta | grep -v motif_alt_id | awk '{print $3"\t"$4-1"\t"$5}' | sort -k1,1 -k2,2n | bedtools merge -i - | cut -f 1 | uniq -c | awk '{print $2"\t"$1}' > t1

grep ">" dmel_to_dtri_dmel_nonconserved_coords_with_bound_id_fasta | sed -r 's/>//' | awk '{print $1"\t0"}' >> t1

cat t1 | sort -k1,1 | bedtools groupby -g 1 -c 2 -o sum | sort -k1b,1 > t1.1

./bin/fimo --thresh 1e-5 --text BEAF32_MA0529.2.meme dmel_to_dtri_dtri_non_boundary_coord_with_bound_id_fasta | grep -v motif_alt_id | awk '{print $3"\t"$4-1"\t"$5}' | sort -k1,1 -k2,2n | bedtools merge -i - | cut -f 1 | sed -r 's/Z[0-9]//' | sort | uniq -c | awk '{print $2"\t"$1}' > t2

grep ">" dmel_to_dtri_dtri_non_boundary_coord_with_bound_id_fasta | sed -r 's/>//' | awk '{print $1"\t0"}' >> t2

cat t2 | sort -k1,1 | bedtools groupby -g 1 -c 2 -o sum | sort -k1b,1 > t2.1

join t1.1 t2.1 | tr " " "\t" > t3

join t3 dmel_to_dtri_dtri_non_boundary_coord_with_bound_id_fasta.len | tr " " "\t" > t4.dmel.beaf


paired_comp.dmel_m1bp.sh

In [ ]:
#find motifs in dmel lineage-specific boundaries and orthologous dtri regions 
./bin/fimo --thresh 1e-5 --text M1BP_MA1459.1.meme /projects/genetics/ellison_lab/nicole/homer/dmel_to_dtri_dmel_nonconserved_coords_with_bound_id_fasta | grep -v motif_alt_id | awk '{print $3"\t"$4-1"\t"$5}' | sort -k1,1 -k2,2n | bedtools merge -i - | cut -f 1 | uniq -c | awk '{print $2"\t"$1}' > t1

grep ">" /projects/genetics/ellison_lab/nicole/homer/dmel_to_dtri_dmel_nonconserved_coords_with_bound_id_fasta | sed -r 's/>//' | awk '{print $1"\t0"}' >> t1

cat t1 | sort -k1,1 | bedtools groupby -g 1 -c 2 -o sum | sort -k1b,1 > t1.1

./bin/fimo --thresh 1e-5 --text M1BP_MA1459.1.meme dmel_to_dtri_dtri_non_boundary_coord_with_bound_id_fasta.rename | grep -v motif_alt_id | awk '{print $3"\t"$4-1"\t"$5}' | sort -k1,1 -k2,2n | bedtools merge -i - | cut -f 1 | sed -r 's/Z[0-9]//' | sort | uniq -c | awk '{print $2"\t"$1}' > t2

grep ">" /projects/genetics/ellison_lab/nicole/homer/dmel_to_dtri_dtri_non_boundary_coord_with_bound_id_fasta | sed -r 's/>//' | awk '{print $1"\t0"}' >> t2

cat t2 | sort -k1,1 | bedtools groupby -g 1 -c 2 -o sum | sort -k1b,1 > t2.1

join t1.1 t2.1 | tr " " "\t" > t3

join t3 dmel_to_dtri_dtri_non_boundary_coord_with_bound_id_fasta.len | tr " " "\t" > t4.dmel.m1bp